<a href="https://colab.research.google.com/github/kingsgameBen/iShape_tools/blob/main/pornstars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import unicodedata
from datetime import datetime
# 判斷是否可以轉換成數字(若字串為'-'，則回傳False)
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
    try:
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

# 計算年齡(只用年月計算，不考慮日期)
def calculate_age(birth):
    birth_d = datetime.strptime(birth, "%Y-%m")
    today_d = datetime.now()
    if today_d.month> birth_d.month:
        age = today_d.year - birth_d.year
    else:
        age = today_d.year - birth_d.year - 1
    return age

### 從Kaggle上找到的日本女優資料集，
因為缺少體重資訊，故從網上尋找有體重資料的女優，並篩選出來，
再尋找是否有正面全身照

In [ ]:
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from urllib.request import urlopen, Request, HTTPError
from random import choice
import shutil
# from selenium.webdriver import Chrome
# from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import NoSuchElementException
import time
import ssl
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
ssl._create_default_https_context = ssl._create_unverified_context

av_dn = "drive/MyDrive/body_shape/"
av_csv_fn = av_dn + "actress.csv"
# print(av_csv_fn)
av_csv = pd.read_csv(av_csv_fn, encoding="utf-8", usecols=[0,1,3,4,6,7,8])
print(len(av_csv))
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width',100)
av_df = pd.DataFrame(av_csv)
# print(av_df.dropna(axis=0, thresh=7))
filter_1 = av_df.dropna(axis=0,how="any", subset=["waist","hips"])
print(len(av_df.dropna(axis=0, subset=["height"])))
print(len(filter_1))

# print(filter_1["height"])
# print(len(av_df["bust"].dropna()))
# print(len(av_df.dropna(axis=0, subset=["bust","waist","hips"])))
# filter_1 = filter_1.dropna(axis=0, subset=["height"]) # 過濾沒身高的資料

name_list = filter_1["name"]
for na in name_list:
    na = na.strip()

av_df["weight"] = np.nan
for idx, row in av_df.iterrows():
    if not pd.isnull(row["birthday"]):
        av_df.loc[idx, "age"] = round(calculate_age(row["birthday"][:-3]))

# print(type(av_df.loc[0]["age"]))
print("從當前女優名 拿到 id 的值", av_df[av_df.name=="JULIA"].id.values[0])
# print("從當前女優名 拿到 weight 的值\n", av_df[av_df.index == av_df[av_df.name=="JULIA"].index[0]])
# av_df.loc[av_df[av_df.name=="JULIA"].index[0], "weight"]=0
print(av_df[av_df.name=="JULIA"])
print(name_list[:50])

### 從維基百科爬取有體重資料的女優

In [ ]:
import random
base_url = "https://zh.wikipedia.org/wiki/"
count = 0
filtered_count = 0
agents = [
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.14 (KHTML, like Gecko) Chrome/24.0.1292.0 Safari/537.14",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1290.1 Safari/537.13",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1290.1 Safari/537.13",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1290.1 Safari/537.13",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_4) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1290.1 Safari/537.13",
"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.13 (KHTML, like Gecko) Chrome/24.0.1284.0 Safari/537.13",
"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.6 Safari/537.11",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.6 Safari/537.11",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.26 Safari/537.11",
"Mozilla/5.0 (Windows NT 6.0) yi; AppleWebKit/345667.12221 (KHTML, like Gecko) Chrome/23.0.1271.26 Safari/453667.1221",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.17 Safari/537.11",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.4 (KHTML, like Gecko) Chrome/22.0.1229.94 Safari/537.4",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_0) AppleWebKit/537.4 (KHTML, like Gecko) Chrome/22.0.1229.79 Safari/537.4",
"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"
]
for name in name_list:
    if len(name.split("（")) > 1:
        other_names = name.split("（")
        other_names[-1] = other_names[-1][:-1]
        if "，" in other_names[-1]:
            aliases = name.split("（")[-1].split("，")
            other_names = [name.split("（")[0], aliases[0], aliases[-1][:-1]]
    else:
        other_names = [name]
    for i_name in other_names:
        url = base_url + i_name
        h ={
            "user-agent": random.choice(agents)
        } 
        response = requests.get(url, headers=h).text
        html = BeautifulSoup(response, features="html.parser")
        main_wrapper = html.find("div", id="mw-content-text")
        infobox = main_wrapper.find("table", class_="infobox")
        if infobox:
            infos = infobox.find("tbody").select("tr")
            if "體重" in infobox.find("tbody").text or "体重" in infobox.find("tbody").text:
                count += 1
                # print("第", count, "筆：", name)
                for info in infos:
                    if "公斤" in info.text:
                        wei_info = info.text.split("/")[-1].strip()[:-2]
                        # print("第", count, "筆：", info.text)
                        if is_number(wei_info):
                            filtered_count += 1
                            av_df.loc[av_df[av_df.name == name].index[0], "weight"] = wei_info
                            # print("有體重的第", filtered_count, "筆：", "weight:", wei_info)
                            print("recheck:", av_df[av_df.name == name]["weight"])
    # print("="*100, "\n")
print(filtered_count)

### 輸出女優體重、腰圍、臀圍資料 -> .csv

In [ ]:
import numpy as np
# hei = info.text.split("/")[1].split(" ")[0]
                    # print("身高：", hei)

# av_df = av_df["weight"].replace("", np.NaN)
# print(av_df.dropna(axis=0, subset=["weight"]))
av_df.dropna(axis=0, how='any', subset=["waist","hips","weight"], inplace=True)
# print(np.any(av_df.isnull())==True)
print(av_df[~av_df.isna()])
print(len(av_df[~av_df.isna()]))
# print(av_df.dropna(axis=0, subset=["bust","waist","hips","weight"]))
# print(len(av_df.dropna(axis=0, subset=["bust","waist","hips","weight"])))
# av_df.dropna(axis=0, subset=["weight"])
# av_df.to_csv(av_dn + "avinfo.csv")
av_df.to_csv(av_dn + "avinfo3.csv")
    # with open(av_csv, "w") as txtf:
    #     txtf.write(details)